the original features ideal is from https://www.kaggle.com/code/taylorsamarel/hybrid-solution-nfl-big-data
in this notebook i've train my version of stacking model including:

- Catboost
- lgb
- xgb
  
 with tuned parameters

# Config

In [ ]:
class Config:
    train_mode=True # True for training, False for testing
    test_mode =True
    pretrained_model_path='/kaggle/input/nfl-2025-tree-stack/other/5folds/2'
    # keep_train_features=['game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y', 'final_pre_throw_x', 'final_pre_throw_y', 's', 'a', 'o', 'dir', 'num_frames_output', 'ball_land_x', 'ball_land_y', 'target_receiver_x', 'target_receiver_y', 'absolute_yardline_number', 'height_inches', 'player_weight', 'time_seconds', 'normalized_frame', 'x_std', 'ball_x_std', 'ball_y_std', 'trg_x_std', 'frame_offset', 'time_offset', 't_rel', 'distance_to_ball_landing', 'angle_to_ball_landing', 'ball_dx', 'ball_dy', 'ball_horizontal_distance', 'ball_vertical_distance', 'estimated_time_to_ball', 'distance_speed_ratio', 'distance_to_target', 'angle_to_target', 'is_target_receiver', 'is_quarterback', 'is_receiver', 'is_offensive_line', 'target_speed_last', 'velocity_x', 'velocity_y', 'speed_y', 'momentum_x', 'momentum_y', 'acceleration_magnitude', 'is_decelerating', 'speed_orientation_discrepancy', 'orientation_alignment', 'vel_toward_ball', 'vel_toward_target', 'distance_from_sideline_x', 'distance_from_sideline_y', 'is_offense', 'is_defense', 'bmi', 'qb_x_std', 'qb_y_std', 'distance_to_qb', 'min_distance_to_any', 'avg_distance_to_any', 'players_within_3y', 'players_within_5y', 'players_within_10y', 'closest_teammate_distance', 'closest_opponent_distance', 'teammates_within_3y', 'teammates_within_5y', 'teammates_within_10y', 'opponents_within_10y', 'teammates_ahead', 'opponents_ahead', 'ball_dist_rank', 'ball_dist_rank_pct', 'target_dist_rank_offense', 'target_dist_rank_offense_pct', 'lateral_position_importance', 'downfield_progress', 'speed_times_acceleration', 'downfield_velocity', 'displacement_x', 'displacement_y']
    ids_cols = ['game_id', 'play_id', 'nfl_id', 'frame_id']
    base_features = [
    'velocity_x_lag1',
    'momentum_y',
    'time_x_speed',
    'velocity_x',
    'time_squared_x_speed',
    'velocity_y',
    'play_direction',
    'ball_dy',
    'velocity_y_lag1',
    'downfield_velocity',
    'ball_direction_x',
    'momentum_x',
    'ball_vertical_distance',
    'ball_direction_y',
    'time_x_acceleration',
    'distance_to_ball_x_time_squared',
    'ball_dx',
    'downfield_progress',
    'time_ratio_to_ball',
    'angle_to_ball_landing',
    'distance_to_target_x_time',
    'target_speed_last',
    'o',
    'teammates_within_10y',
    'angle_to_target',
    'ball_horizontal_distance',
    'dir',
    'teammates_ahead',
    'vel_toward_ball',
    'closing_speed',
    'time_squared',
    'vel_toward_target',
    'speed_times_acceleration',
    'normalized_frame',
    'frame_offset',
    'a',
    'distance_to_target',
    'distance_to_ball_x_time',
    'projected_time_to_ball',
    'speed_orientation_discrepancy',
    'bmi',
    'ball_land_y',
    'lateral_position_importance',
    'distance_from_sideline_y',
    'acceleration_magnitude',
    'time_urgency',
    'orientation_alignment',
    'ball_y_std',
    'time_seconds',
    'players_within_10y',
    'speed_lag1',
    'expected_y_with_accel',
    'y_lag1',
    'player_weight',
    'target_receiver_y',
    'distance_speed_ratio',
    'proximity_to_ball_ratio',
    'log_time',
    'momentum_magnitude',
    'closest_opponent_distance',
    't_rel',
    'teammates_within_5y',
    'ball_land_x',
    'trg_y_std',
    'ball_x_std',
    'expected_y_constant_v',
    'players_within_5y',
    'players_within_3y',
    'distance_to_ball_landing',
    'separation_advantage',
    'kinetic_energy',
    'opponents_ahead',
    'time_cubed',
    'ball_dist_rank',
    'distance_from_sideline_x',
    'sqrt_time',
    'trg_x_std',
    'target_receiver_x',
    'expected_x_constant_v',
    'ball_dist_rank_pct',
    'x_std',
    'opponents_within_10y',
    'x_lag1',
    'expected_x_with_accel',
    'estimated_time_to_ball',
    'distance_from_endzone',
    'absolute_yardline_number',
    'teammates_within_3y',
    'y_last',
    'height_inches',
    'opponents_within_3y',
    'yards_to_endzone',
    'time_sin',
    's',
    'speed_rolling_mean_3',
    'opponents_within_5y',
    'is_late_play',
    'T',
    'speed_squared',
    'num_frames_output',
    'final_pre_throw_y',
    'normalized_y',
    'time_cos',
    'y_std',
    'in_own_territory',
    'final_pre_throw_x']
    # no (['displacement_x', 'displacement_y','x','y'])
    target_features = ['displacement_x', 'displacement_y','x','y']
    keep_train_features= ids_cols + base_features + target_features
    keep_test_features=ids_cols + base_features.copy()
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    MAX_SPEED = 12.0
    N_FOLDS = 5
    NN_BATCH_SIZE = 2048
    NN_EPOCHS = 30
    NN_LEARNING_RATE = 0.001

In [ ]:
cfg = Config()

# NFL Big Data Bowl 2026 - Data Loading and Preprocessing


## Loading Dataset

This section handles loading and combining the weekly training data, test input, and test template files.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
import os
import xgboost as xgb
warnings.filterwarnings('ignore')

In [ ]:
def load_multiple_csv_files(file_paths):
    """Load and concatenate multiple CSV files"""
    data_frames = [pd.read_csv(p) for p in file_paths]
    return pd.concat(data_frames, ignore_index=True)

In [ ]:
def load_and_combine_datasets(data_dir, weeks=range(1, 18)):
    """Load and combine weekly training data, test input, and test template"""
    data_path = Path(data_dir)

    # Load training input data
    input_paths = [data_path / f"train/input_2023_w{w:02d}.csv" for w in weeks]
    train_input = load_multiple_csv_files(input_paths)

    # Load training output data
    output_paths = [data_path / f"train/output_2023_w{w:02d}.csv" for w in weeks]
    train_output = load_multiple_csv_files(output_paths)

    # Load test data
    test_input = pd.read_csv(data_path / "test_input.csv")
    test_template = pd.read_csv(data_path / "test.csv")

    return train_input, train_output, test_input, test_template

## Preprocess Data

This section handles data preprocessing including extracting final pre-throw observations, incorporating target receiver data, and calculating advanced features.

In [ ]:
def convert_height_to_inches(height_str):
    """Convert height from 'ft-in' format to total inches"""
    if not isinstance(height_str, str) or '-' not in height_str:
        return np.nan
    try:
        feet, inches = map(int, height_str.split('-'))
        return feet * 12 + inches
    except (ValueError, AttributeError):
        return np.nan

In [ ]:
def extract_final_pre_throw_observation(tracking_data):
    """Extract the last tracking frame before pass for each player"""
    # Check available columns
    print("Available columns in tracking data:", tracking_data.columns.tolist())

    # Ensure we have the required grouping columns
    required_cols = ['game_id', 'play_id', 'nfl_id', 'frame_id']
    missing_cols = [col for col in required_cols if col not in tracking_data.columns]
    if missing_cols:
        raise KeyError(f"Missing required columns: {missing_cols}")

    sorted_data = tracking_data.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    final_observation = sorted_data.groupby(['game_id', 'play_id', 'nfl_id'], as_index=False).last()

    # Rename position columns for clarity
    if 'x' in final_observation.columns:
        final_observation = final_observation.rename(columns={'x': 'final_pre_throw_x'})
    if 'y' in final_observation.columns:
        final_observation = final_observation.rename(columns={'y': 'final_pre_throw_y'})

    # Convert height to inches if available
    if 'player_height' in final_observation.columns:
        final_observation['height_inches'] = final_observation['player_height'].apply(
            convert_height_to_inches
        )

    return final_observation

In [ ]:
def incorporate_target_receiver_data(player_data):
    """Add target receiver position data to all players in the same play"""
    # Check if player_role column exists
    if 'player_role' not in player_data.columns:
        print("Warning: 'player_role' column not found. Skipping target receiver incorporation.")
        player_data['target_receiver_x'] = np.nan
        player_data['target_receiver_y'] = np.nan
        return player_data

    target_receivers = player_data[player_data['player_role'] == "Targeted Receiver"][
        ['game_id', 'play_id', 'final_pre_throw_x', 'final_pre_throw_y']
    ].rename(columns={
        'final_pre_throw_x': 'target_receiver_x',
        'final_pre_throw_y': 'target_receiver_y'
    })

    # Remove duplicates if any
    target_receivers = target_receivers.drop_duplicates(['game_id', 'play_id'])

    return player_data.merge(target_receivers, on=['game_id', 'play_id'], how='left')

In [ ]:
def calculate_advanced_features(data_frame, training_mode=False):
    """Create comprehensive feature set with advanced football analytics"""
    df = data_frame.copy()

    # Check available columns and create features accordingly
    available_columns = df.columns.tolist()
    print("Available columns for feature engineering:", available_columns)

    # Helpers
    def has_cols(cols):
        return all(col in df.columns for col in cols)

    # Time-based features
    if 'frame_id' in df.columns:
        df['time_seconds'] = df['frame_id'] / 10.0  # 10 FPS
        frame_max = df.groupby(['game_id', 'play_id', 'nfl_id'])['frame_id'].transform('max')
        df['normalized_frame'] = df['frame_id'] / frame_max.replace(0, np.nan)

    # Standardized orientation (mirror so offense always moves right)
    # This keeps y unchanged because play_direction is left/right along x-axis.
    if has_cols(['final_pre_throw_x', 'final_pre_throw_y', 'play_direction']):
        pd_right = (df['play_direction'].str.lower() == 'right')
        df['x_std'] = np.where(pd_right, df['final_pre_throw_x'], 120.0 - df['final_pre_throw_x'])
        df['y_std'] = df['final_pre_throw_y']
        if has_cols(['ball_land_x', 'ball_land_y']):
            df['ball_x_std'] = np.where(pd_right, df['ball_land_x'], 120.0 - df['ball_land_x'])
            df['ball_y_std'] = df['ball_land_y']
        if has_cols(['target_receiver_x', 'target_receiver_y']):
            df['trg_x_std'] = np.where(pd_right, df['target_receiver_x'], 120.0 - df['target_receiver_x'])
            df['trg_y_std'] = df['target_receiver_y']
    else:
        # Fallback to raw positions as standardized
        if 'final_pre_throw_x' in df.columns:
            df['x_std'] = df['final_pre_throw_x']
        if 'final_pre_throw_y' in df.columns:
            df['y_std'] = df['final_pre_throw_y']
        if has_cols(['ball_land_x', 'ball_land_y']):
            df['ball_x_std'] = df['ball_land_x']
            df['ball_y_std'] = df['ball_land_y']
        if has_cols(['target_receiver_x', 'target_receiver_y']):
            df['trg_x_std'] = df['target_receiver_x']
            df['trg_y_std'] = df['target_receiver_y']

    # New feature: expose last pre-throw aliases to match other pipeline
    if has_cols(['final_pre_throw_x', 'final_pre_throw_y']):
        df['x_last'] = df['final_pre_throw_x']
        df['y_last'] = df['final_pre_throw_y']

    # New feature: time-offset features aligned with output frames (10 fps)
    if has_cols(['frame_id', 'num_frames_output']):
        df['frame_offset'] = df['frame_id'].astype(float)
        # avoid division by zero
        df['T'] = df['num_frames_output'].astype(float).clip(lower=1.0)
        df['t_rel'] = df['frame_offset'] / df['T']

    # Ball trajectory features
    if has_cols(['ball_x_std', 'ball_y_std', 'x_std', 'y_std']):
        dx = df['ball_x_std'] - df['x_std']
        dy = df['ball_y_std'] - df['y_std']
        dist = np.hypot(dx, dy)
        df['distance_to_ball_landing'] = dist
        df['angle_to_ball_landing'] = np.arctan2(dy, dx)
        # Components
        df['ball_dx'] = dx
        df['ball_dy'] = dy
        df['ball_horizontal_distance'] = np.abs(dx)
        df['ball_vertical_distance'] = np.abs(dy)
        # Time proxy
        if 's' in df.columns:
            df['estimated_time_to_ball'] = dist / (df['s'].fillna(0) + 0.1)
            df['distance_speed_ratio'] = dist / (df['s'].fillna(0) + 1.0)
            df['proximity_to_ball_ratio'] = dist / (df['s'].fillna(0) + 0.1)

    # Target receiver features
    if has_cols(['trg_x_std', 'trg_y_std', 'x_std', 'y_std']):
        tdx = df['trg_x_std'] - df['x_std']
        tdy = df['trg_y_std'] - df['y_std']
        df['distance_to_target'] = np.hypot(tdx, tdy)
        df['angle_to_target'] = np.arctan2(tdy, tdx)

    # Target indicator and broader roles
    if 'player_role' in df.columns:
        pr = df['player_role'].astype(str)
        df['is_target_receiver'] = (pr == "Targeted Receiver").astype(int)
        # Coarse role buckets
        df['is_quarterback'] = pr.str.contains(r'\bQB\b', case=False, regex=True).astype(int)
        df['is_receiver'] = pr.str.contains(r'WR|TE|RB|Receiver', case=False, regex=True).astype(int)
        df['is_offensive_line'] = pr.str.contains(r'\bC\b|\bG\b|\bT\b|OL', case=False, regex=True).astype(int)
        df['is_passer'] = pr.str.contains(r'\bPasser\b', case=False, regex=True).astype(int)
        df['is_coverage'] = pr.str.contains(r'Defensive Coverage', case=False, regex=True).astype(int)
    else:
        df['is_target_receiver'] = 0

    # New feature: targeted receiver speed and position per play (from last pre-throw snapshot)
    if has_cols(['game_id', 'play_id', 'player_role']):
        cols_to_take = ['game_id', 'play_id']
        if 's' in df.columns:
            cols_to_take.append('s')
        if 'player_position' in df.columns:
            cols_to_take.append('player_position')
        trg = df.loc[df['player_role'].astype(str) == 'Targeted Receiver', cols_to_take].copy()
        if 's' in trg.columns:
            trg = trg.rename(columns={'s': 'target_speed_last'})
        if 'player_position' in trg.columns:
            trg = trg.rename(columns={'player_position': 'target_position'})
        trg = trg.groupby(['game_id', 'play_id'], as_index=False).first()
        df = df.merge(trg, on=['game_id', 'play_id'], how='left')

    # Velocity components
    if has_cols(['s', 'dir']):
        dir_rad = np.deg2rad(df['dir'])
        df['velocity_x'] = df['s'] * np.sin(dir_rad)
        df['velocity_y'] = df['s'] * np.cos(dir_rad)
        df['speed_squared'] = (df['s'] ** 2)
        df['is_moving'] = (df['s'] > 0.5).astype(int)

    # New feature: momentum components (weight in kg; fallback 90.7 kg)
    if has_cols(['velocity_x', 'velocity_y']):
        if 'player_weight' in df.columns:
            mass_kg = (df['player_weight'].fillna(200.0) / 2.20462)
        else:
            mass_kg = 90.7  # ~200 lbs
        df['momentum_x'] = df['velocity_x'] * mass_kg
        df['momentum_y'] = df['velocity_y'] * mass_kg

    # Acceleration features
    if 'a' in df.columns:
        df['acceleration_magnitude'] = np.abs(df['a'])
        df['is_accelerating'] = (df['a'] > 0.5).astype(int)
        df['is_decelerating'] = (df['a'] < -0.5).astype(int)

    # Heading vs facing
    if has_cols(['dir', 'o']):
        # Proper circular difference in degrees
        ang_diff = np.abs(df['dir'] - df['o'])
        ang_diff = np.where(ang_diff > 180.0, 360.0 - ang_diff, ang_diff)
        df['speed_orientation_discrepancy'] = ang_diff
        df['orientation_alignment'] = np.cos(np.deg2rad(ang_diff))

    # Radial velocity components toward ball/target
    if has_cols(['velocity_x', 'velocity_y', 'ball_dx', 'ball_dy']):
        mag = np.hypot(df['ball_dx'], df['ball_dy']).replace(0, np.nan)
        ux = df['ball_dx'] / mag
        uy = df['ball_dy'] / mag
        df['vel_toward_ball'] = (df['velocity_x'] * ux + df['velocity_y'] * uy).fillna(0.0)
    if has_cols(['velocity_x', 'velocity_y', 'distance_to_target', 'trg_x_std', 'trg_y_std', 'x_std', 'y_std']):
        tdx = df['trg_x_std'] - df['x_std']
        tdy = df['trg_y_std'] - df['y_std']
        tmag = np.hypot(tdx, tdy).replace(0, np.nan)
        tux = tdx / tmag
        tuy = tdy / tmag
        df['vel_toward_target'] = (df['velocity_x'] * tux + df['velocity_y'] * tuy).fillna(0.0)

    # Field position features
    if 'final_pre_throw_x' in df.columns:
        df['normalized_x'] = df['final_pre_throw_x'] / 120.0
        df['field_region_x'] = pd.cut(df['final_pre_throw_x'], bins=6, labels=False)
        df['distance_from_sideline_x'] = np.minimum(df['final_pre_throw_x'], 120.0 - df['final_pre_throw_x'])
        df['in_redzone'] = (df['final_pre_throw_x'] >= 100.0).astype(int)
        df['in_own_territory'] = (df['final_pre_throw_x'] <= 50.0).astype(int)

    if 'final_pre_throw_y' in df.columns:
        df['normalized_y'] = df['final_pre_throw_y'] / 53.3
        df['field_region_y'] = pd.cut(df['final_pre_throw_y'], bins=4, labels=False)
        df['distance_from_sideline_y'] = np.minimum(df['final_pre_throw_y'], 53.3 - df['final_pre_throw_y'])
        df['near_sideline'] = (df['distance_from_sideline_y'] <= 5.0).astype(int)

    # Game context features
    if 'absolute_yardline_number' in df.columns:
        df['yards_to_endzone'] = df['absolute_yardline_number']
        # thirds of field
        df['field_position_thirds'] = pd.cut(df['absolute_yardline_number'],
                                             bins=[-0.1, 33, 67, 100.1], labels=[0, 1, 2]).astype('Int64')

    # Team indicator
    if 'player_side' in df.columns:
        df['is_offense'] = (df['player_side'] == 'Offense').astype(int)
        df['is_defense'] = (df['player_side'] == 'Defense').astype(int)
    else:
        df['is_offense'] = 0
        df['is_defense'] = 0

    # Player physical attributes
    if has_cols(['player_weight', 'height_inches']):
        valid_height = df['height_inches'] > 0
        df['bmi'] = np.nan
        df.loc[valid_height, 'bmi'] = (df.loc[valid_height, 'player_weight'] * 0.453592) / (
            (df.loc[valid_height, 'height_inches'] * 0.0254) ** 2
        )
        df['weight_category'] = pd.cut(df['player_weight'],
                                       bins=[0, 200, 250, 300, float('inf')],
                                       labels=[0, 1, 2, 3]).astype('Int64')
        df['height_category'] = pd.cut(df['height_inches'],
                                       bins=[0, 70, 74, 78, float('inf')],
                                       labels=[0, 1, 2, 3]).astype('Int64')

    # Team-level QB position (from roles) and distances
    if has_cols(['game_id', 'play_id', 'player_role', 'x_std', 'y_std']):
        pr = df['player_role'].astype(str)
        qb_mask = pr.str.contains(r'\bQB\b|Passer', case=False, regex=True)
        # Pick one QB per play if exists
        qb_df = df.loc[qb_mask, ['game_id', 'play_id', 'x_std', 'y_std']].copy()
        qb_df = qb_df.groupby(['game_id', 'play_id'], as_index=False).first().rename(
            columns={'x_std': 'qb_x_std', 'y_std': 'qb_y_std'}
        )
        df = df.merge(qb_df, on=['game_id', 'play_id'], how='left')
        if has_cols(['qb_x_std', 'qb_y_std', 'x_std', 'y_std']):
            df['distance_to_qb'] = np.hypot(df['qb_x_std'] - df['x_std'], df['qb_y_std'] - df['y_std'])

    # Density and separation features per play (vectorized, no group apply pitfalls)
    if has_cols(['game_id', 'play_id', 'x_std', 'y_std']):
        for (gid, pid), idx in df.groupby(['game_id', 'play_id']).groups.items():
            inds = np.array(list(idx))
            xs = df.loc[inds, 'x_std'].to_numpy(dtype=float)
            ys = df.loc[inds, 'y_std'].to_numpy(dtype=float)
            coords = np.stack([xs, ys], axis=1)  # [N, 2]
            N = coords.shape[0]

            # Pairwise distances
            d2 = np.sum((coords[:, None, :] - coords[None, :, :])**2, axis=2)  # [N, N]
            # avoid self
            np.fill_diagonal(d2, np.inf)
            d = np.sqrt(d2)

            # Closest teammate/opponent distances if sides available
            closest_any = np.min(d, axis=1)
            df.loc[inds, 'min_distance_to_any'] = closest_any
            df.loc[inds, 'avg_distance_to_any'] = np.where(N > 1, np.mean(d, axis=1), np.nan)

            # Counts within radii
            for r, name in [(3.0, '3y'), (5.0, '5y'), (10.0, '10y')]:
                counts = (d <= r).sum(axis=1)
                df.loc[inds, f'players_within_{name}'] = counts

            if 'player_side' in df.columns:
                side = df.loc[inds, 'player_side'].astype(str).to_numpy()
                is_off = (side == 'Offense')
                is_def = (side == 'Defense')

                # teammate/opponent masks
                team_mask = (is_off[:, None] & is_off[None, :]) | (is_def[:, None] & is_def[None, :])
                oppo_mask = (is_off[:, None] & is_def[None, :]) | (is_def[:, None] & is_off[None, :])

                # Set self distances to inf again in these masked matrices
                d_team = d.copy()
                d_oppo = d.copy()
                d_team[~team_mask] = np.inf
                d_oppo[~oppo_mask] = np.inf
                # self index inf remains

                df.loc[inds, 'closest_teammate_distance'] = np.min(d_team, axis=1)
                df.loc[inds, 'closest_opponent_distance'] = np.min(d_oppo, axis=1)
                df.loc[inds, 'separation_advantage'] = df.loc[inds, 'closest_opponent_distance'] - df.loc[inds, 'closest_teammate_distance']

                # Counts within radii by class
                for r, name in [(3.0, '3y'), (5.0, '5y'), (10.0, '10y')]:
                    df.loc[inds, f'teammates_within_{name}'] = ((d_team <= r).sum(axis=1)).astype(int)
                    df.loc[inds, f'opponents_within_{name}'] = ((d_oppo <= r).sum(axis=1)).astype(int)

                # Linear "ahead" counts (downfield along x_std)
                x = xs
                teammates_ahead = np.zeros(N, dtype=int)
                opponents_ahead = np.zeros(N, dtype=int)
                for i in range(N):
                    if is_off[i] or is_def[i]:
                        if is_off[i]:
                            teammates_ahead[i] = int(((is_off) & (x > x[i])).sum())
                            opponents_ahead[i] = int(((is_def) & (x > x[i])).sum())
                        else:
                            teammates_ahead[i] = int(((is_def) & (x > x[i])).sum())
                            opponents_ahead[i] = int(((is_off) & (x > x[i])).sum())
                df.loc[inds, 'teammates_ahead'] = teammates_ahead
                df.loc[inds, 'opponents_ahead'] = opponents_ahead

            # Rank by distance to ball within play
            if has_cols(['ball_x_std', 'ball_y_std']):
                bdx = df.loc[inds, 'ball_x_std'].to_numpy(dtype=float) - xs
                bdy = df.loc[inds, 'ball_y_std'].to_numpy(dtype=float) - ys
                dist_ball = np.hypot(bdx, bdy)
                # percent rank (smaller is better)
                order = dist_ball.argsort()
                rank = np.empty_like(order)
                rank[order] = np.arange(N)
                df.loc[inds, 'ball_dist_rank'] = rank
                df.loc[inds, 'ball_dist_rank_pct'] = np.where(N > 1, rank / (N - 1), np.nan)

            # Rank to target receiver among offense
            if has_cols(['trg_x_std', 'trg_y_std']) and 'player_side' in df.columns:
                off_idx = np.where(df.loc[inds, 'player_side'] == 'Offense')[0]
                if off_idx.size > 0:
                    tdx = df.loc[inds, 'trg_x_std'].to_numpy(dtype=float) - xs
                    tdy = df.loc[inds, 'trg_y_std'].to_numpy(dtype=float) - ys
                    off_dist = np.hypot(tdx[off_idx], tdy[off_idx])
                    order = off_dist.argsort()
                    rank = np.empty_like(order)
                    rank[order] = np.arange(off_idx.size)
                    # default NaN for defense
                    trg_rank = np.full(N, np.nan)
                    trg_rank_pct = np.full(N, np.nan)
                    trg_rank[off_idx] = rank
                    trg_rank_pct[off_idx] = np.where(off_idx.size > 1, rank / (off_idx.size - 1), np.nan)
                    df.loc[inds, 'target_dist_rank_offense'] = trg_rank
                    df.loc[inds, 'target_dist_rank_offense_pct'] = trg_rank_pct

    # Advanced football-specific features
    if 'final_pre_throw_y' in df.columns:
        df['lateral_position_importance'] = np.abs(df['final_pre_throw_y'] - 26.65) / 26.65

    if has_cols(['final_pre_throw_x', 'yards_to_endzone']):
        df['downfield_progress'] = df['final_pre_throw_x'] - df['yards_to_endzone']

    # Interaction features
    if has_cols(['s', 'a']):
        df['speed_times_acceleration'] = df['s'] * df['a']

    # Direction-adjusted velocity along field (downfield component)
    if has_cols(['velocity_x', 'play_direction']):
        pd_right = (df['play_direction'].str.lower() == 'right')
        # velocity_x is positive when moving to increasing x (to the right in raw coords)
        df['downfield_velocity'] = np.where(pd_right, df['velocity_x'], -df['velocity_x'])

    # New: Temporal and Sequential Features
    # Ensure data is sorted (already done earlier in the function)
    if has_cols(['game_id', 'play_id', 'nfl_id', 'frame_id']):
        group_cols = ['game_id', 'play_id', 'nfl_id']
        # Lagged features (previous frame)
        if 'velocity_x' in df.columns:
            df['velocity_x_lag1'] = df.groupby(group_cols)['velocity_x'].shift(1)
        if 'velocity_y' in df.columns:
            df['velocity_y_lag1'] = df.groupby(group_cols)['velocity_y'].shift(1)
        if 's' in df.columns:
            df['speed_lag1'] = df.groupby(group_cols)['s'].shift(1)
        if 'final_pre_throw_x' in df.columns:
            df['x_lag1'] = df.groupby(group_cols)['final_pre_throw_x'].shift(1)
        if 'final_pre_throw_y' in df.columns:
            df['y_lag1'] = df.groupby(group_cols)['final_pre_throw_y'].shift(1)
        
        # Rolling statistics (e.g., mean over last 3 frames) - Fixed with transform
        if 's' in df.columns:
            df['speed_rolling_mean_3'] = df.groupby(group_cols)['s'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
        if 'a' in df.columns:
            df['acceleration_rolling_std_3'] = df.groupby(group_cols)['a'].transform(lambda x: x.rolling(window=3, min_periods=1).std())
    # New: Physics-Based Features
    if has_cols(['player_weight', 's']):
        mass_kg = df['player_weight'] / 2.20462  # Convert lbs to kg
        df['kinetic_energy'] = 0.5 * mass_kg * (df['s'] ** 2)  # 0.5 * m * v^2
    if has_cols(['momentum_x', 'momentum_y']):
        df['momentum_magnitude'] = np.hypot(df['momentum_x'], df['momentum_y'])  # Magnitude of momentum vector
    # Time-based polynomial and cyclical features (build on existing time_seconds and normalized_frame)
    if 'time_seconds' in df.columns:
        df['time_squared'] = df['time_seconds'] ** 2
        df['time_cubed'] = df['time_seconds'] ** 3
        df['sqrt_time'] = np.sqrt(df['time_seconds'])
        df['log_time'] = np.log1p(df['time_seconds'])
        
        # Fourier features for cyclical patterns
        if 'normalized_frame' in df.columns:
            df['time_sin'] = np.sin(2 * np.pi * df['normalized_frame'])
            df['time_cos'] = np.cos(2 * np.pi * df['normalized_frame'])
            df['time_sin_2'] = np.sin(4 * np.pi * df['normalized_frame'])
            df['time_cos_2'] = np.cos(4 * np.pi * df['normalized_frame'])
            
            # Phase-based features
            df['is_early_play'] = (df['normalized_frame'] < 0.33).astype(int)
            df['is_mid_play'] = ((df['normalized_frame'] >= 0.33) & (df['normalized_frame'] < 0.67)).astype(int)
            df['is_late_play'] = (df['normalized_frame'] >= 0.67).astype(int)
    # Physics-based expected positions (build on velocity_x, velocity_y, a, dir)
    if has_cols(['velocity_x', 'velocity_y', 'time_seconds', 'x_last', 'y_last']):
        df['expected_x_constant_v'] = df['x_last'] + df['velocity_x'] * df['time_seconds']
        df['expected_y_constant_v'] = df['y_last'] + df['velocity_y'] * df['time_seconds']
        
        if 'a' in df.columns and 'dir' in df.columns:
            dir_rad = np.deg2rad(df['dir'])
            df['expected_x_with_accel'] = df['x_last'] + df['velocity_x'] * df['time_seconds'] + \
                                        0.5 * df['a'] * np.sin(dir_rad) * df['time_seconds']**2
            df['expected_y_with_accel'] = df['y_last'] + df['velocity_y'] * df['time_seconds'] + \
                                        0.5 * df['a'] * np.cos(dir_rad) * df['time_seconds']**2

    # Movement consistency features (build on s_mean, a_mean from temporal stats; assume you add temporal stats below)
    if has_cols(['s', 's_mean', 'a', 'a_mean']):
        df['speed_consistency'] = df['s'] / (df['s_mean'] + 0.1)
        df['speed_deviation'] = np.abs(df['s'] - df['s_mean'])
        df['acceleration_consistency'] = df['a'] / (df['a_mean'] + 0.1)
        df['acceleration_deviation'] = np.abs(df['a'] - df['a_mean'])

    # Temporal interaction features
    if has_cols(['time_seconds', 's', 'a']):
        df['time_x_speed'] = df['time_seconds'] * df['s']
        df['time_x_acceleration'] = df['time_seconds'] * df['a']
        df['time_squared_x_speed'] = df['time_seconds']**2 * df['s']
    # Ball-related features (enhance existing distance_to_ball_landing, angle_to_ball_landing)
    if has_cols(['distance_to_ball_landing', 'ball_dx', 'ball_dy', 'time_seconds', 'velocity_x', 'velocity_y']):
        # Unit vectors
        dist = df['distance_to_ball_landing']
        df['ball_direction_x'] = df['ball_dx'] / (dist + 1e-6)
        df['ball_direction_y'] = df['ball_dy'] / (dist + 1e-6)
        
        # Time-based
        estimated_ball_speed = 20.0  # Assumed ball speed
        df['estimated_time_to_ball'] = dist / estimated_ball_speed
        df['time_ratio_to_ball'] = df['time_seconds'] / (df['estimated_time_to_ball'] + 0.1)
        
        # Closing speed and urgency
        df['closing_speed'] = df['velocity_x'] * df['ball_direction_x'] + df['velocity_y'] * df['ball_direction_y']
        df['projected_time_to_ball'] = dist / (np.abs(df['closing_speed']) + 0.1)
        df['time_urgency'] = df['time_seconds'] / (df['projected_time_to_ball'] + 0.1)
        
        # Interactions
        df['distance_to_ball_x_time'] = dist * df['time_seconds']
        df['distance_to_ball_x_time_squared'] = dist * df['time_seconds']**2

    # Target receiver features (enhance existing distance_to_target, angle_to_target)
    if has_cols(['player_role', 'distance_to_target', 'time_seconds']):
        df['distance_to_target_x_time'] = df['distance_to_target'] * df['time_seconds']
        df['is_target_x_time_squared'] = df['is_target_receiver'] * df['time_seconds']**2

    # Field position features (enhance existing normalized_x, normalized_y)
    if has_cols(['x_last', 'y_last']):
        df['distance_from_endzone'] = np.minimum(df['x_last'], 120.0 - df['x_last'])
    if training_mode and has_cols(['x', 'final_pre_throw_x', 'y', 'final_pre_throw_y']):
        df['displacement_x'] = df['x'] - df['final_pre_throw_x']
        df['displacement_y'] = df['y'] - df['final_pre_throw_y']
        # New feature: extra displacement diagnostics (optionally drop before training)
        # df['displacement_magnitude'] = np.hypot(df['displacement_x'], df['displacement_y'])
        # df['displacement_angle'] = np.arctan2(df['displacement_y'], df['displacement_x'])

    # Clip any infinities created by empty groups
    num_cols = df.select_dtypes(include=[np.number]).columns
    for c in num_cols:
        df[c] = df[c].replace([np.inf, -np.inf], np.nan)

    return df

In [ ]:
def prepare_features(input_data, output_data, training_mode=False):
    """Complete feature engineering pipeline"""
    print("Extracting final pre-throw observations...")
    final_observations = extract_final_pre_throw_observation(input_data)
    print("Incorporating target receiver data...")
    final_observations = incorporate_target_receiver_data(final_observations)

    # Identify available columns for merging
    available_columns = final_observations.columns.tolist()
    merge_columns = ['game_id', 'play_id', 'nfl_id']

    # Add other columns if they exist
    optional_columns = [
        'final_pre_throw_x', 'final_pre_throw_y', 's', 'a', 'o', 'dir',
        'player_role', 'player_side', 'num_frames_output', 'ball_land_x',
        'ball_land_y', 'target_receiver_x', 'target_receiver_y',
        'play_direction', 'absolute_yardline_number', 'height_inches', 'player_weight'
    ]

    for col in optional_columns:
        if col in available_columns:
            merge_columns.append(col)

    print(f"Merging with columns: {merge_columns}")

    # Merge with output data
    merged_data = output_data.merge(
        final_observations[merge_columns],
        on=['game_id', 'play_id', 'nfl_id'],
        how='left'
    )

    print("Calculating advanced features...")
    return calculate_advanced_features(merged_data, training_mode=training_mode)

## Preproc

In [ ]:
# Example usage


print("Loading datasets...")
data_dir = "/kaggle/input/nfl-big-data-bowl-2026-prediction/"
train_input, train_output, test_input, test_template = load_and_combine_datasets(data_dir)

print(f"Train input shape: {train_input.shape}")
print(f"Train output shape: {train_output.shape}")
print(f"Test input shape: {test_input.shape}")
print(f"Test template shape: {test_template.shape}")


In [ ]:
if (cfg.train_mode == True):
    print("Preparing training features...")
    train_features = prepare_features(train_input, train_output, training_mode=True)
    train_features = train_features[cfg.keep_train_features]
    print(f"Train features shape: {train_features.shape}")
# train_features = encode_categorical_features(train_features, ['player_position', 'player_side', 'play_direction'])

if (cfg.test_mode == True):
    print("Preparing test features...")
    test_features = prepare_features(test_input, test_template, training_mode=False)
    test_features = test_features[cfg.keep_test_features]
    print(f"Test features shape: {test_features.shape}")
id_cols = ['game_id', 'play_id', 'nfl_id', 'frame_id']

# Metric

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error


class ParticipantVisibleError(Exception):
    pass





def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Compute RMSE for NFL competition.
    Expected input:
      - solution and submission as pandas.DataFrame
      - Column 'id': unique identifier for each (game_id, play_id, nfl_id, frame_id)
      - Column 'x'
      - Column 'y'
    Examples
    --------
    >>> import pandas as pd
    >>> row_id_column_name = 'id'
    >>> solution = pd.DataFrame({'id': ['21_12_2_1', '21_12_2_2', '21_12_2_3'], 'x': [1,2,3], 'y':[4,2,3]})
    >>> submission  = pd.DataFrame({'id': ['21_12_2_1', '21_12_2_2', '21_12_2_3'], 'x': [1.1,2,3], 'y':[4,2.2,3]})
    >>> round(score(solution, submission, row_id_column_name=row_id_column_name), 4)
    0.0913
    >>> submission  = pd.DataFrame({'id': ['21_12_2_1', '21_12_2_2', '21_12_2_3'], 'x': [0,2,3], 'y':[4,2.2,3]})
    >>> round(score(solution, submission, row_id_column_name=row_id_column_name), 4)
    0.4163
    >>> submission  = pd.DataFrame({'id': ['21_12_2_1', '21_12_2_2', '21_12_2_3'], 'x': [1,2,1], 'y':[4,0,3]})
    >>> round(score(solution, submission, row_id_column_name=row_id_column_name), 4)
    1.1547
    """
    TARGET = ['x', 'y']
    if row_id_column_name not in solution.columns:
        raise ParticipantVisibleError(f"Solution file missing required column: '{row_id_column_name}'")
    if row_id_column_name not in submission.columns:
        raise ParticipantVisibleError(f"Submission file missing required column: '{row_id_column_name}'")

    missing_in_solution = set(TARGET) - set(solution.columns)
    missing_in_submission = set(TARGET) - set(submission.columns)

    if missing_in_solution:
        raise ParticipantVisibleError(f'Solution file missing required columns: {missing_in_solution}')
    if missing_in_submission:
        raise ParticipantVisibleError(f'Submission file missing required columns: {missing_in_submission}')

    submission = submission[['id'] + TARGET]
    merged_df = pd.merge(solution, submission, on=row_id_column_name, suffixes=('_true', '_pred'))
    rmse = np.sqrt(
        0.5 * (mean_squared_error(merged_df['x_true'], merged_df['x_pred']) + mean_squared_error(merged_df['y_true'], merged_df['y_pred']))
    )
    return float(rmse)

In [ ]:
def make_submission(model, data, id_cols, target_cols, label_encoders, categorical_columns,
                    is_validation=False, val_true=None, predict_displacement=True):
    if is_validation:
        X = data.drop(columns=target_cols + id_cols+['x', 'y'])
    else:
        X = data.drop(columns=id_cols)

    for col in categorical_columns:
        if col in X.columns and col in label_encoders:
            X[col] = X[col].astype(str).fillna('Unknown')
            X[col] = X[col].where(X[col].isin(label_encoders[col].classes_), 'Unknown')
            X[col] = label_encoders[col].transform(X[col])

    preds = model.predict(X)
    if preds.ndim == 1:
        preds = np.column_stack([preds, np.zeros_like(preds)])

    submission = data[id_cols + ['final_pre_throw_x','final_pre_throw_y']].copy()
    if predict_displacement:
        submission['x'] = submission['final_pre_throw_x'] + preds[:, 0]
        submission['y'] = submission['final_pre_throw_y'] + preds[:, 1]
    else:
        submission['x'] = preds[:, 0]
        submission['y'] = preds[:, 1]

    # clip to field bounds
    submission['x'] = submission['x'].clip(0.0, 120.0)
    submission['y'] = submission['y'].clip(0.0, 53.3)

    submission['id'] = (
        submission['game_id'].astype(str) + '_' +
        submission['play_id'].astype(str) + '_' +
        submission['nfl_id'].astype(str) + '_' +
        submission['frame_id'].astype(str)
    )

    submission = submission[['id', 'x', 'y']]
    if is_validation and val_true is not None:
        val_true1 = val_true[['id', 'x', 'y']]
        print(val_true1.head())
        print(submission.head())
        rmse = score(val_true1, submission, 'id')
        print(f"Validation RMSE: {rmse}")
        return rmse, submission

    return submission
# ...existing code...

# split correctly

In [ ]:
from sklearn.model_selection import GroupKFold

# Group by game_id + play_id to avoid leakage
groups = train_features['game_id'].astype(str) + '_' + train_features['play_id'].astype(str)
gkf = GroupKFold(n_splits=5)
train_idx, val_idx = next(gkf.split(train_features, groups=groups))
train_data = train_features.iloc[train_idx].copy()
val_data = train_features.iloc[val_idx].copy()

# ...existing code...
TARGET = ['displacement_x', 'displacement_y']
X_train = train_data.drop(columns=TARGET + id_cols + ['x','y'])
y_train = train_data[TARGET]

X_val = val_data.drop(columns=TARGET + id_cols + ['x','y'])
y_val = val_data[TARGET]
print(f"Training set shape: {X_train.shape}, {y_train.shape}")
from sklearn.preprocessing import LabelEncoder
categorical_columns = ['player_role', 'player_side', 'play_direction']

# Encode categorical features
label_encoders = {}
for col in categorical_columns:
    if col in X_train.columns:
        le = LabelEncoder()
        X_train[col] = le.fit_transform(X_train[col].astype(str))
        # X_val[col] = le.transform(X_val[col].astype(str))
        label_encoders[col] = le

# trees stacking

## Params

In [ ]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.multioutput import MultiOutputRegressor
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Define base models with updated parameters
cat_params = {
    'iterations': 1926,
    'learning_rate': 0.04387999210613504,
    'depth': 10,
    'random_seed': 42,
    'verbose': False,
    'task_type': 'CPU' if device == 'cpu' else 'GPU',
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE'
}

lgb_params = {
    'n_estimators': 784,
    'learning_rate': 0.036023311815726135,
    'max_depth': 10,
    'num_leaves': 139,
    'subsample': 0.8515467775473755,
    'colsample_bytree': 0.6666089092760663,
    'random_state': 42,
    'objective': 'regression',
    'device_type': 'cpu' if device == 'cpu' else 'gpu',
    'verbose': -1
}

xgb_params = {
    'n_estimators': 1997,
    'learning_rate': 0.019246218249748592,
    'max_depth': 10,
    'subsample': 0.8780270231551571,
    'colsample_bytree': 0.6416701179410014,
    'random_state': 42,
    'tree_method': 'gpu_hist' if device == 'cuda' else 'hist',
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'enable_categorical': False
}

# 5 folds training

In [ ]:
# backup_train_features = train_features.copy()

In [ ]:
# train_features = backup_train_features.copy()
# train_features.shape

In [ ]:
# ...existing code...
#import HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import GroupKFold
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler
# Group by game_id + play_id to avoid leakage
# train_features = train_features.sample(frac=0.00009, random_state=42)
if cfg.train_mode:
    groups = train_features['game_id'].astype(str) + '_' + train_features['play_id'].astype(str)
    gkf = GroupKFold(n_splits=5)
    test_submission_list = []
    fold_scores = []

    for fold,(train_idx, val_idx) in enumerate(gkf.split(train_features, groups=groups)):
        
        
        train_data = train_features.iloc[train_idx].copy()
        
        val_data = train_features.iloc[val_idx].copy()
        print(train_data.shape, val_data.shape)
        # ...existing code...
        TARGET = ['displacement_x', 'displacement_y']
        X_train = train_data.drop(columns=TARGET + id_cols + ['x','y'])
        y_train = train_data[TARGET]
        
        X_val = val_data.drop(columns=TARGET + id_cols + ['x','y'])
        y_val = val_data[TARGET]
        
        categorical_columns = ['player_role', 'player_side', 'play_direction']

        # Encode categorical features
        label_encoders = {}
        for col in categorical_columns:
            if col in X_train.columns:
                le = LabelEncoder()
                X_train[col] = le.fit_transform(X_train[col].astype(str))
                # X_val[col] = le.transform(X_val[col].astype(str))
        #         label_encoders[col] = le
        # imputer = SimpleImputer(strategy='mean')
        # X_train = imputer.fit_transform(X_train)
        # X_val = imputer.transform(X_val)
        base_models = [
        ('cat', CatBoostRegressor(**cat_params)),
        ('lgb', LGBMRegressor(**lgb_params)),
        ('xgb', xgb.XGBRegressor(**xgb_params))
        ]

        meta_model = HistGradientBoostingRegressor()

        stacking_model = StackingRegressor(
            estimators=base_models,
            final_estimator=meta_model,
            passthrough=True,
            n_jobs=1
        )
        fold_dir = f'fold_{fold}'
        os.makedirs(fold_dir, exist_ok=True)
        multi_stacking_model = MultiOutputRegressor(stacking_model)
        print(f'Training fold {fold}...')
        multi_stacking_model.fit(X_train, y_train)
        joblib.dump(multi_stacking_model, f'fold_{fold}/multi_stacking_model.pkl')
        #save label encoders
        joblib.dump(label_encoders, f'fold_{fold}/label_encoders.pkl')

        #save columns
        joblib.dump(X_train.columns.tolist(), f'fold_{fold}/model_columns.pkl')
        val_true = val_data[['game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y']].copy()
        val_true['id'] = (
            val_true['game_id'].astype(str) + '_' +
            val_true['play_id'].astype(str) + '_' +
            val_true['nfl_id'].astype(str) + '_' +
            val_true['frame_id'].astype(str)
        )
        fold_score,val_submission = make_submission(
            model=multi_stacking_model,
            data=val_data,
            id_cols=id_cols,
            target_cols=TARGET,
            label_encoders=label_encoders,
            categorical_columns=categorical_columns,
            is_validation=True,
            val_true=val_true
        )

        test_submission = make_submission(
            model=multi_stacking_model,
            data=test_features,
            id_cols=id_cols,
            target_cols=TARGET,
            label_encoders=label_encoders,
            categorical_columns=categorical_columns,
            is_validation=False
        )
        print(f'Fold {fold} RMSE: {fold_score}')
        fold_scores.append(fold_score)
        test_submission_list.append(test_submission)
        #print top 20 features importance of each base model in the stacking model
        # for name, model in multi_stacking_model.estimators_[0].estimators:
        #     if name == 'cat':
        #         importances = model.get_feature_importance()
        #     else:
        #         importances = model.feature_importances_
        #     importance_df = pd.DataFrame({'feature': X_train.columns, 'importance': importances}).sort_values('importance', ascending=False).head(20)
        #     plt.figure(figsize=(10, 6))
        #     plt.barh(importance_df['feature'], importance_df['importance'])
        #     plt.xlabel('Importance')
        #     plt.ylabel('Feature')
        #     plt.title(f'Top 20 Feature Importances for {name} in Fold {fold}')
        #     plt.gca().invert_yaxis()
        #     plt.show()
    
    #average fold scores
    print(f'Average fold RMSE: {np.mean(fold_scores)}')
    for i, score in enumerate(fold_scores):
        print(f'Fold {i} RMSE: {score}')

# making submission

# Loading model for infer


In [ ]:
if cfg.test_mode == True:
    
    if cfg.train_mode == False:
        print("Loading pretrained models...")
        models_dir = cfg.pretrained_model_path
    else:
        models_dir = '/kaggle/working'
    test_submission_list = []
    for fold in range(5):
        fold_dir = models_dir+f'/fold_{fold}'
        multi_stacking_model = joblib.load(fold_dir+'/multi_stacking_model.pkl')
        label_encoders = joblib.load(fold_dir+'/label_encoders.pkl')
        model_columns = joblib.load(fold_dir+'/model_columns.pkl')
        categorical_columns = ['player_role', 'player_side', 'play_direction']
        TARGET = ['displacement_x', 'displacement_y']
        
        # val_true = val_data[['game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y']].copy()
        # val_true['id'] = (
        #     val_true['game_id'].astype(str) + '_' +
        #     val_true['play_id'].astype(str) + '_' +
        #     val_true['nfl_id'].astype(str) + '_' +
        #     val_true['frame_id'].astype(str)
        # )
        # fold_score,val_submission = make_submission(
        #     model=multi_stacking_model,
        #     data=val_data,
        #     id_cols=id_cols,
        #     target_cols=TARGET,
        #     label_encoders=label_encoders,
        #     categorical_columns=categorical_columns,
        #     is_validation=True,
        #     val_true=val_true
        # )
        
        test_submission = make_submission(
            model=multi_stacking_model,
            data=test_features,
            id_cols=id_cols,
            target_cols=TARGET,
            label_encoders=label_encoders,
            categorical_columns=categorical_columns,
            is_validation=False
        )
        test_submission_list.append(test_submission)

In [ ]:
#average the test submissions
final_test_submission = pd.concat(test_submission_list).groupby('id').mean().reset_index()
final_test_submission.to_csv('submission.csv', index=False)

In [ ]:
final_test_submission.head()

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/sample_submission.csv')
#match rows
assert final_test_submission['id'].isin(sample_submission['id']).all() 